# Main flow

- Take each useful news in news.tsv
- joint title and text
- Sample an enhancer and apply to a news. Take the same enhancer and apply it to the generated article to decrease the enhancement (thin of it as a knob that decreases the power of the enhancement).
- Sample a topic change. Extract the main actors from the original news in a first prompt, in the second one generater an article about the actors+topics 

## Post-processing

- Split the generated articles between title and text.
- Store them in separate news files.

In [ ]:
import random


In [ ]:
useful_news['full_text'] = useful_news['Title'].fillna(" ")+" "+useful_news['Abstract'].fillna(" ")

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer

#model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

outputs = model.generate(inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:

# topics pool

topic_sample = ["Political and economic developments: Political and economic developments in key markets can have a significant impact on the global economy and financial markets. For example, changes in government policies, trade agreements, and geopolitical tensions can affect the value of currencies, commodity prices, and stock markets",
 "Natural disasters: Natural disasters such as hurricanes, earthquakes, and floods can cause significant damage to infrastructure, disrupt supply chains, and impact the economy. This can lead to increased volatility in financial markets and affect the value of investments.Include ONLY one of the possible events given. ",
"Health crises: Health crises such as pandemics can have a significant impact on the global economy and financial markets. For example, the COVID-19 pandemic has caused widespread disruptions to global supply chains, reduced demand for certain goods and services, and led to increased volatility in financial markets.",
"Regulatory Changes: Sudden changes in financial regulations can affect how banks operate and their profitability. For instance, the introduction of the Dodd-Frank Act after the 2008 financial crisis had significant implications for investment banks.",

"Cyber Attacks: While companies and governments take precautions against these, a significant cyber attack on a major institution or infrastructure could have far-reaching impacts on the market.",
"Environmental issues: Environmental issues such as climate change and resource depletion can have a significant impact on the global economy and financial markets. For example, changes in weather patterns and resource availability can affect agricultural production, energy prices, and the value of investments.Include ONLY one of the possible events given",
"Trade Wars and Tariffs: Sudden escalations in trade wars or the imposition of new tariffs are events that can impact global trade and consequently, the financial markets.Include ONLY one of the possible events given",
"Sudden Corporate Bankruptcies: The collapse of a major company or financial institution, such as the bankruptcy of Lehman Brothers in 2008, can have a ripple effect on the economy and financial markets."

]


In [ ]:
#enhancement pools

enhancers_sample = ["The level of detail and specificity provided in the article. One article may provide more in-depth information about the event, including its causes, effects, and potential solutions. Please generate specific data, figures, numbers and details  that are not included in the article please do so.",
"The credibility and reliability of the sources cited in the article. An article that cites reputable sources, such as experts in the field or government agencies, may be more helpful for an investment banker as it provides a more reliable basis for making informed decisions.",
"Monetary Impact: The degree of impact an event has can often be measured by the scale of monetary value involved. The larger the amount of money implicated, either in terms of cost, revenue, investment, or loss, the more significant the event can be considered.  Please generate specific figures. ",
"Number of stakeholders impacted: The larger the number of stakeholders affected, the more profound the potential implications. This can range from employees and shareholders to customers, suppliers and residents. Include ONLY one of the examples and quantify the impact in terms of number of people",


]

In [ ]:
def apply_enhancement(article, i =None):

    if i is None:
        i= random.randrange(len(enhancers_sample))

    # PROMPT TEMPLATE
    messages = [

    {"role": "user", "content": f" We need to generate plausible news articles from examples. Given the following news article  {article}. \n Re-write it so as to increase and enhance {enhancers_sample[i]}. "}
]

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

    outputs = model.generate(inputs, max_new_tokens=len(article)+100)

    llm_out = tokenizer.decode(outputs[0], skip_special_tokens=True)
    llm_out = llm_out.split("[/INST]")[1]


    return llm_out , i 


    

In [ ]:
def apply_impairment(article, i =None):

    if i is None:

        i= random.randrange(len(enhancers_sample))

    # PROMPT TEMPLATE
    messages = [

    {"role": "user", "content": f" We need to generate plausible news articles from examples. Given the following news article \n {article}. \n Re-write and change figures so as to greatly lessen or decrease the {enhancers_sample[i]}. "}
    ]   

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

    outputs = model.generate(inputs, max_new_tokens=len(article)+50)

    llm_out = tokenizer.decode(outputs[0], skip_special_tokens=True)
    llm_out = llm_out.split("[/INST]")[1]


    return llm_out ,i

In [ ]:
import re

In [ ]:
def apply_topic_change2(article, i =None):


    messages = [
        
    # {"role": "assistant", "content": "You are a helpful and truthful AI agent."},


    # PROMPT TEMPLATE
        {"role": "user", "content": f" Given the following article  {article}. \n Extract only the most important companies', organizations', persons' and places' names in the article .  \n Please do not copy full sentences. Please return only the most important companies, organizations, people and places in the article and nothing else. Return only proper nouns and separate them with commas. "}
    ]
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

    outputs = model.generate(inputs, max_new_tokens=12,temperature=1.0)

    llm_out = tokenizer.decode(outputs[0], skip_special_tokens=True)
    protagonists = llm_out.split("[/INST]")[1]

    protagonists = protagonists.rstrip(",")
    #print(protagonists)
    
    #protagonists = ",".join([noun  for noun in protagonists.split(",") if noun.lstrip()[0].isupper()])
    #re.search("[A-Z]","ipad")
    protagonists = ",".join([noun  for noun in protagonists.split(",") if re.search("[A-Z]",noun.lstrip()) is not None])


    #print(protagonists)

    if i is None:
        i= random.randrange(len(topic_sample))


    messages = [
        
    # {"role": "assistant", "content": "You are a helpful and truthful AI agent."},
        {"role": "user", "content": f" We need to generate plausible news articles from examples. Given the following protagonist  companies, organizations, people and geographies: \n  {protagonists}, \n choose any of them and create a new coherent article that deals with the following topic: {topic_sample[i]}.  \n The first sentence of the new article should be the title, and should mention the companies, organizations, geographies or people selected before. Please do not copy full sentences from the topic description and only use it as a guide. If you need to generate specific data, numbers and details  that are not included in the article please do so. Please only return the new article."}
    ]

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

    outputs = model.generate(inputs, max_new_tokens=len(article)+100,temperature=0.95)

    
    llm_out = tokenizer.decode(outputs[0], skip_special_tokens=True)
    llm_out = llm_out.split("[/INST]")[1]
    llm_out = llm_out.replace("Title: ", "")

    return llm_out ,i

In [ ]:
enhancement_ls = []
impairment_ls = []
unusual_topic_ls2 = []

for article in useful_news.loc[~useful_news['already_done']].iloc[16650:17450]["full_text"]:

    synt_new, i = apply_enhancement(article)
    enhancement_ls.append((synt_new, i))


    impairment_ls.append(apply_impairment(synt_new,i))

    unusual_topic_ls2.append(apply_topic_change2(article))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/felipe/anaconda3/envs/cuda_12.1/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable 

In [ ]:
len(enhancement_ls)

800